In [1]:
import torch
import torch_geometric

/home/icb/vnicolas.goedert/miniconda3/envs/RFAM/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rfam_dir = "../rfam/data/raw/processed/release-14.8"
rfams = ["RF00001","RF00005","RF00169","RF00050", "RF00020"]

from RNARepLearn.datasets import CombinedRfamDataset, SingleRfamDataset
#dataset = CombinedRfamDataset(rfam_dir, rfams, "Under300", 15, 300)
#dataset = CombinedRfamDataset(rfam_dir, ["RF00001","RF00005"], "RF00001_RF00005")
#dataset = SingleRfamDataset(rfam_dir,"RF00001")
dataset = CombinedRfamDataset(rfam_dir, rfams, "Under1000_RF00001_RF00005_RF00169_RF00050_RF00020", 1000)

from RNARepLearn.utils import train_val_test_loaders

train_loader, val_loader, test_loader = train_val_test_loaders(dataset, 0.8, 0.01, 0.19)

Processing complete
Training:	19085
Test:	238
Validation:	4534


Processing...
Done!


In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("runs/Masked2501")

In [3]:
from RNARepLearn.modules import LinearEmbedding, RPINetEncoder, AttentionDecoder
layers = []
layers.append(RPINetEncoder(4, 32, 3, 3))
layers.append(AttentionDecoder(32, 4))
model = torch.nn.Sequential(*layers)

In [4]:
from RNARepLearn.train import MaskedTraining, AutoEncoder
training = MaskedTraining(model, 3, 15, writer)
#training = AutoEncoder(model, 3, 15)

NameError: name 'writer' is not defined

In [ ]:
training.run(train_loader, val_loader)

In [9]:
from RNARepLearn.modules import GCN_Encoder
batch = next(iter(train_loader))
batch

DataBatch(x=[3624, 4], edge_index=[2, 157000], edge_weight=[157000], rfam=[32], ID=[32], batch=[3624], ptr=[33])

In [10]:
encoder = GCN_Encoder(4,64,9, 5).double()

In [11]:
encoder(batch)

DataBatch(x=[3624, 64], edge_index=[2, 157000], edge_weight=[157000], rfam=[32], ID=[32], batch=[3624], ptr=[33])